# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], int(row[4]), row[5], float(row[6]), row[7], row[8], int(row[12]), row[13], int(row[16])))


8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

# try, except in case of error
try:
    cluster = Cluster()
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

# Queries

For each of the queries that are needed, do the following:
- create table

In [8]:
my_csv = pd.read_csv('event_datafile_new.csv')

## Query1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
# Query artist, song, length
# where session_id == 338, itemInSession == 4
query = "CREATE TABLE IF NOT EXISTS artists "
query = query + "(artist text, itemInSession int, length float, sessionId int, song text, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [10]:
# Directory of the data file 
file = 'event_datafile_new.csv'

# Assigning the INSERT statements into the `query` variable
# The query is the same throughout the loop, doesn't have to be changed.
# Small saving, but still
query = "INSERT INTO artists (artist, itemInSession, length, sessionId, song)"
query = query + " VALUES (%s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assigning which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
sessionId = 338
itemInSession = 4
query = f"select artist, song, length from artists WHERE sessionId={sessionId} AND itemInSession={itemInSession}"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [12]:
# Test, if all makes sense
my_csv[(my_csv.sessionId == sessionId) & (my_csv.itemInSession == itemInSession)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


## Query2: Give me name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
# Query artist, song, firstName, lastName
# where session_id == 182, userId == 10
# order by itemInSession
query = "CREATE TABLE IF NOT EXISTS artist_listened "
query = query + "(artist text, firstName text, lastName text, sessionId int, song text, userId int, itemInSession int, PRIMARY KEY ((sessionId, userId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

query = "INSERT INTO artist_listened (artist, firstName, lastName, sessionId, song, userId, itemInSession)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (line[0], line[1], line[4], int(line[8]), line[9], int(line[10]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
sessionId = 182
userId = 10
query = f"select artist, song, firstName, lastName from artist_listened WHERE sessionId={sessionId} AND userId={userId} ORDER BY itemInSession ASC;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [16]:
# Test, if all makes sense
my_csv[(my_csv.sessionId == sessionId) & (my_csv.userId == userId)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


## Query3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [17]:
# Query firstName, lastName
# where song == 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS songs_listened "
query = query + "(firstName text, lastName text, song text, userId int, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [19]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

query = "INSERT INTO songs_listened (firstName, lastName, song, userId)"
query = query + " VALUES (%s, %s, %s, %s)"


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [20]:
song_name = 'All Hands Against His Own'
query = "select firstName, lastName from songs_listened WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


In [21]:
# Test, if all makes sense
my_csv[(my_csv.song == song_name)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Drop the tables before closing out the sessions

In [46]:
query = "drop table artists"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table artist_listened"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songs_listened"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [47]:
session.shutdown()
cluster.shutdown()